<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/stock_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Utility Functions

In [ ]:
# @title GitHub
import base64
import requests
import pandas as pd
from io import StringIO


class GitHub:
    def __init__(self, repo_owner, repo_name, token, dir='', branch='main'):
        base = 'https://api.github.com/repos'
        dir = dir.strip('/')
        if dir:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch
        self.token = token

    def _request(self, method, url, headers=None, params=None, json=None):
        response = requests.request(method, url, headers=headers,
                                    params=params, json=json)
        if response.status_code in [200, 201]:
            return response.json()
        elif response.status_code == 404:
            return None
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def file_exists(self, file_path):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            return True
        elif response.status_code == 404:
            return False
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def list_filenames(self, dir_path=''):
        url = f'{self.base_url}/{dir_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return []

    def download_file(self, file_path):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }

        file_info = self._request('GET', url, headers=headers,
                                  params={'ref': self.branch})
        if file_info:
            response = requests.get(file_info['download_url'])
            if response.status_code == 200:
                return StringIO(response.text)
            else:
                print(f"Failed to download file: "
                      f"{response.status_code} - {response.text}")
        else:
            print(f"File '{file_path}' does not exist. Cannot download.")
        return None

    def download_csv(self, file_path):
        file_content = self.download_file(file_path)
        if file_content:
            return pd.read_csv(file_content)
        else:
            return pd.DataFrame()

    def upload_file(self, file_path, content):
        url = f'{self.base_url}/{file_path}'

        # Encode the content to base64
        encoded_content = base64.b64encode(content.encode()).decode()
        payload = {
            'message': 'Uploading file',
            'content': encoded_content,
            'branch': self.branch
        }

        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Check if the file already exists to get the current sha
        file_info = self._request('GET', url, headers=headers,
                                  params={'ref': self.branch})
        # If the file exists, get the current SHA
        if file_info:
            payload['sha'] = file_info.get('sha')

        # PUT request to create or update the file
        self._request('PUT', url, headers=headers, json=payload)

    def upload_df_as_csv(self, file_path, df):
        """Upload a DataFrame to a CSV file."""
        if not file_path.endswith('.csv'):
            file_path += '.csv'
        csv_content = df.to_csv(index=False)
        self.upload_file(file_path, csv_content)

    def remove_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Skipping deletion.")
            return

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Fetch the file info to get the SHA needed for deletion
        file_info = self._request('GET', url, headers=headers)
        if file_info:
            payload = {
                'message': 'Deleting file',
                'sha': file_info['sha'],
                'branch': self.branch
            }
            self._request('DELETE', url, headers=headers, json=payload)

#-------------------------------------------------------------------------------

from google.colab import userdata

github = GitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    token=userdata.get('GithubToken.stock-reports'),
    dir='stock_list',
)

In [ ]:
# @title From Wikipedia

import functools
from io import StringIO

import requests
import pandas as pd
from bs4 import BeautifulSoup


def table_from_wikipedia(article,
                         class_='wikitable sortable',  id='constituents'):
    """
    Fetches a table from a Wikipedia article and returns it as a pandas
    DataFrame.

    Args:
        article (str): The name of the Wikipedia article.
        class_ (str): The class attribute of the table to retrieve.
        id (str): The id attribute of the table to retrieve.

    Returns:
        pandas.DataFrame: The retrieved table.
    """
    url = f"https://en.wikipedia.org/wiki/{article}"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    attrs = {}
    if class_:
        attrs['class'] = class_
    if id:
        attrs['id'] = id
    tag = soup.find('table', attrs=attrs)
    return pd.read_html(StringIO(str(tag)))[0]


def symbols_from_wikipedia_table(article,
                                 class_='wikitable sortable',
                                 id='constituents'):
    """
    Extracts stock symbols from a table in a Wikipedia article.

    Args:
        article (str): The name of the Wikipedia article.
        class_ (str, optional): The class attribute of the table. Defaults to
            'wikitable sortable'.
        id (str, optional): The id attribute of the table. Defaults to
            'constituents'.

    Returns:
        list: A list of stock symbols.
    """
    df = table_from_wikipedia(article, class_, id)
    if 'Symbol' in df.columns:
        return df['Symbol'].tolist()
    elif 'Ticker' in df.columns:
        return df['Ticker'].tolist()
    return []


spx_tickers = functools.partial(
        symbols_from_wikipedia_table, 'List_of_S%26P_500_companies')
djia_tickers = functools.partial(
        symbols_from_wikipedia_table, 'Dow_Jones_Industrial_Average')
ndx_tickers = functools.partial(
        symbols_from_wikipedia_table, 'Nasdaq-100')
rui_tickers = functools.partial(
        symbols_from_wikipedia_table, 'Russell_1000_Index', id=None)


In [ ]:
# @title From BullishBears Website

import functools

import pandas as pd


def table_from_bullishbears(article):
    """
    Fetches the first table from a specified Bullish Bears article.

    Args:
        article (str): The relative path of the article on the Bullish Bears
            website.

    Returns:
        pd.DataFrame: The first table found in the specified URL, parsed into
            a Pandas DataFrame.
    """
    url = f'https://bullishbears.com/{article}'
    return pd.read_html(url)[0]


def symbols_from_bullishbears_table(article):
    """
    Extracts stock symbols from a table in a Bullish Bears article.

    Args:
        article (str): The relative path of the article on the Bullish Bears
            website.

    Returns:
        list: A list of stock symbols from the table's "Symbol" column.

    Examples:
        >>> symbols = symbols_from_bullishbears_table('sp500-stocks-list')
        >>> len(symbols)
        503
        >>> symbols = symbols_from_bullishbears_table('dow-jones-stocks-list')
        >>> len(symbols)
        30
        >>> symbols = symbols_from_bullishbears_table('nasdaq-stocks-list')
        >>> len(symbols)
        100
        >>> symbols = symbols_from_bullishbears_table(
        ...     'russell-2000-stocks-list')
        >>> 'SMCI' in symbols
        True
        >>> len(symbols) > 1990
        True
    """
    df = table_from_bullishbears(article)
    return list(df['Symbol'])


rut_tickers = functools.partial(
        symbols_from_bullishbears_table, 'russell-2000-stocks-list')


### Examples

In [ ]:
print('DJIA:', djia_tickers())
print('NDX:', ndx_tickers())
print('SPX:', spx_tickers())
print('RUI:', rui_tickers())
print('RUT:', rut_tickers())

DJIA: ['MMM', 'AXP', 'AMGN', 'AMZN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WMT']
NDX: ['ADBE', 'AMD', 'ABNB', 'GOOGL', 'GOOG', 'AMZN', 'AEP', 'AMGN', 'ADI', 'ANSS', 'AAPL', 'AMAT', 'ARM', 'ASML', 'AZN', 'TEAM', 'ADSK', 'ADP', 'BKR', 'BIIB', 'BKNG', 'AVGO', 'CDNS', 'CDW', 'CHTR', 'CTAS', 'CSCO', 'CCEP', 'CTSH', 'CMCSA', 'CEG', 'CPRT', 'CSGP', 'COST', 'CRWD', 'CSX', 'DDOG', 'DXCM', 'FANG', 'DLTR', 'DASH', 'EA', 'EXC', 'FAST', 'FTNT', 'GEHC', 'GILD', 'GFS', 'HON', 'IDXX', 'ILMN', 'INTC', 'INTU', 'ISRG', 'KDP', 'KLAC', 'KHC', 'LRCX', 'LIN', 'LULU', 'MAR', 'MRVL', 'MELI', 'META', 'MCHP', 'MU', 'MSFT', 'MRNA', 'MDLZ', 'MDB', 'MNST', 'NFLX', 'NVDA', 'NXPI', 'ORLY', 'ODFL', 'ON', 'PCAR', 'PANW', 'PAYX', 'PYPL', 'PDD', 'PEP', 'QCOM', 'REGN', 'ROP', 'ROST', 'SBUX', 'SMCI', 'SNPS', 'TTWO', 'TMUS', 'TSLA', 'TXN', 'TTD', 'VRSK', 'VRTX', 'WBD', 'WDAY', 'XEL', 'ZS']
SP

In [ ]:
df = table_from_wikipedia('List_of_S%26P_500_companies')
github.upload_df_as_csv('sp500_Wikipedia.csv', df)
df = table_from_wikipedia('Dow_Jones_Industrial_Average')
github.upload_df_as_csv('djia_Wikipedia.csv', df)
df = table_from_wikipedia('Nasdaq-100')
github.upload_df_as_csv('ndx_Wikipedia.csv', df)
df = table_from_wikipedia('Russell_1000_Index', id=None)
github.upload_df_as_csv('rui_Wikipedia.csv', df)
df = table_from_bullishbears('sp500-stocks-list')
github.upload_df_as_csv('sp500_BullishBears.csv', df)
df = table_from_bullishbears('dow-jones-stocks-list')
github.upload_df_as_csv('djia_BullishBears.csv', df)
df = table_from_bullishbears('nasdaq-stocks-list')
github.upload_df_as_csv('ndx_BullishBears.csv', df)
df = table_from_bullishbears('russell-2000-stocks-list')
github.upload_df_as_csv('rui_BullishBears.csv', df)